In [1]:
from colossus.cosmology import cosmology
cosmo=cosmology.setCosmology('planck18')
import numpy as np
import Func
from cobaya.run import run
from getdist import loadMCSamples, plots

# data and fid

In [2]:
zz = np.array([0.15,0.45,0.75])
# error of (f*sigma8) at zz
err_rsd = np.array([0.037622781639503125,0.013120288894571081,0.00801293154199452])
err_rsd_ksz = np.array([0.0048570460593981195,0.0029804102835748773,0.0028959660223023967])

In [3]:
# Fiducial values  
omegam_fid = 0.31655132
sigma8_fid = 0.8119776

fs8 = Func.growthrate(Om0=omegam_fid, z=zz) * Func.sigma(sigma8=sigma8_fid, Om0=omegam_fid, z=zz)
ref=[omegam_fid,sigma8_fid]

In [4]:
Func.growthrate(Om0=omegam_fid, z=0.),Func.sigma(sigma8=sigma8_fid, Om0=omegam_fid, z=0.)

(array(0.52970696), 0.8119776)

In [5]:
fs8

array([0.46022862, 0.47699105, 0.4583774 ])

# RSD

In [6]:
def da(z_array, omegam):
    results = []
    for z in z_array:
        int_z = np.linspace(0, z, 50)
        E = Func.Ez(int_z, omegam)
        integral = np.trapz(1./E, int_z)
        results.append(integral)
    return np.array(results)

def log_like1(omegam,sigma8):

    E = Func.Ez(zz, omegam)
    E_fid = Func.Ez(zz, omegam_fid)

    D = da(zz, omegam)
    D_fid = da(zz, omegam_fid)

    f_th = Func.growthrate(Om0=omegam,z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=omegam,z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi


info = {"likelihood": {"loglike": log_like1}, \
        "params": {"omegam": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[0],"latex": r'\Omega_m'}, \
                    "sigma8": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[1],"latex": r'\sigma8'}}, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd/rsd"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd', with prefix 'rsd'
[output] Found existing info files with the requested output prefix: 'chains_rsd/rsd'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: omegam:0.3165513, sigma8:0.8119776
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 418.0}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-08-30 16:23:38 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 80 samples accepted.
[mcmc]  - Acceptance rate: 0.038
[mcmc]  - Convergence of means: R-1 = 2.989808 aft

In [7]:
readsamps1 = loadMCSamples('chains_rsd/rsd')
cov_rsd = readsamps1.cov(['omegam','sigma8'])
print("cov is:")
print(cov_rsd)
fisher_rsd = np.linalg.inv(cov_rsd)
print("fisher is:")
np.array(fisher_rsd)

[root] *WARNING* outlier fraction 0.01810344827586207 
cov is:
[[ 0.00796255 -0.00472651]
 [-0.00472651  0.00318018]]
fisher is:


array([[1066.30029574, 1584.77749668],
       [1584.77749668, 2669.80634617]])

# RSD and kSZ

In [8]:
def log_like2(omegam,sigma8):

    E = Func.Ez(zz, omegam)
    E_fid = Func.Ez(zz, omegam_fid)

    D = da(zz, omegam)
    D_fid = da(zz, omegam_fid)
    
    f_th = Func.growthrate(Om0=omegam,z=zz)
    sigma8_th = Func.sigma(sigma8=sigma8, Om0=omegam,z=zz)
    
    Chi_1 = (fs8 - (E*D)/(E_fid*D_fid)*sigma8_th*f_th)

    Chi_2 = err_rsd_ksz**2
    Chi_C = Chi_1**2/Chi_2
    Chi = np.sum(Chi_C)
    
    return -0.5*Chi


info = {"likelihood": {"loglike": log_like2}, \
        "params": {"omegam": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[0],"latex": r'\Omega_m'}, \
                    "sigma8": {"prior": {"min": 0.0, "max": 1.0},'ref': ref[1],"latex": r'\sigma8'}}, \
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 10000},},\
        "output": "chains_rsd_ksz/rsd_ksz"
        }
# ,"proposal": 0.001
updated_info, sampler = run(info,force=True)

[output] Output to be read-from/written-into folder 'chains_rsd_ksz', with prefix 'rsd_ksz'
[output] Found existing info files with the requested output prefix: 'chains_rsd_ksz/rsd_ksz'
[output] Will delete previous products ('force' was requested).
[loglike] Initialized external likelihood.
[mcmc] Getting initial point... (this may take a few seconds)
[mcmc] Initial point: omegam:0.3165513, sigma8:0.8119776
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {loglike: 365.0}
[prior] *WARNING* Reference pdf not defined or improper for some parameters. Using prior's sigma instead for them.
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Sampling!
[mcmc] Progress @ 2024-08-30 16:24:07 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 80 samples accepted.
[mcmc]  - Acceptance rate: 0.013
[mcmc]  - Convergence of means: R-

In [9]:
readsamps2 = loadMCSamples('chains_rsd_ksz/rsd_ksz')
cov_rsd_ksz = readsamps2.cov(['omegam','sigma8'])
print("cov is:")
print(cov_rsd_ksz)
fisher_rsd_ksz = np.linalg.inv(cov_rsd_ksz)
print("fisher is:")
np.array(fisher_rsd_ksz)

[root] *WARNING* outlier fraction 0.03409090909090909 
cov is:
[[ 0.00024907 -0.00018977]
 [-0.00018977  0.00015457]]
fisher is:


array([[ 62173.54993919,  76331.72969844],
       [ 76331.72969844, 100183.3901806 ]])

# joint

In [5]:
import numpy as np
from getdist import loadMCSamples, plots
import matplotlib.pyplot as plt

def rgba(cc):
    return tuple(cc/255.)
c1 = rgba(np.array([41.,157.,143.,255.]))
c2 = rgba(np.array([233.,196.,106.,255.]))
c3 = rgba(np.array([216.,118.,89.,255.]))

g=plots.get_subplot_plotter(chain_dir=[
                                       '/home/wangsy2/mcmc/ori/chains_rsd',
                                       '/home/wangsy2/mcmc/ori/chains_rsd_ksz',
                                       '/home/wangsy2/Planck_2018_chains/base/plikHM_TTTEEE_lowl_lowE',
                                       ],
                                       width_inch=7)
roots = ['rsd','rsd_ksz','base_plikHM_TTTEEE_lowl_lowE']
params = ['sigma8','omegam']

g.settings.figure_legend_frame = False
g.settings.alpha_filled_add=0.8
g.settings.title_limit_fontsize = 17

g.triangle_plot(roots, params, filled=True, legend_labels=['RSD','RSD+kSZ+FRB', 'Planck'],
                contour_colors=[c1,c3,c2], param_limits={'sigma8':[0.6, 1.0], 'omegam':[0.1, 0.6]})
g.settings.legend_fontsize = 17


for ax in g.subplots[:, :].flat:
    if ax is not None:  # 检查 ax 是否为 None
        ax.minorticks_on()
        ax.tick_params(which='both', direction='in', top=True, right=True)
        ax.tick_params(which='minor', length=4)
        ax.tick_params(which='major', length=7)
        ax.set_


g.export('joint_new.pdf')


In [3]:
import numpy as np
from getdist import loadMCSamples, plots
import matplotlib.pyplot as plt

def rgba(cc):
    return tuple(cc/255.)
c1 = rgba(np.array([41.,157.,143.,255.]))
c2 = rgba(np.array([233.,196.,106.,255.]))
c3 = rgba(np.array([216.,118.,89.,255.]))

g = plots.get_subplot_plotter(
    chain_dir=[
        '/home/wangsy2/mcmc/ori/chains_rsd',
        '/home/wangsy2/mcmc/ori/chains_rsd_ksz',
        '/home/wangsy2/Planck_2018_chains/base/plikHM_TTTEEE_lowl_lowE',
    ],
    width_inch=7
)

roots = ['rsd', 'rsd_ksz', 'base_plikHM_TTTEEE_lowl_lowE']
params = ['sigma8', 'omegam']

# 设置全局字体参数
g.settings.figure_legend_frame = False
g.settings.alpha_filled_add = 0.8
g.settings.title_limit_fontsize = 16

# 调整字体大小设置
g.settings.axes_labelsize = 18    # 坐标轴标签（如 sigma8, omegam）的字体大小
#g.settings.tick_labelsize = 16    # 坐标轴刻度数字的字体大小
g.settings.legend_fontsize = 14  # 图例字体大小

# 生成三角形图
g.triangle_plot(
    roots, 
    params, 
    filled=True, 
    legend_labels=['RSD', 'RSD+kSZ', 'Planck'],
    contour_colors=[c1, c3, c2], 
    param_limits={'sigma8': [0.6, 1.0], 'omegam': [0.1, 0.6]}
)

# 进一步调整刻度样式
for ax in g.subplots[:, :].flat:
    if ax is not None:
        ax.minorticks_on()
        # 设置刻度方向并调整标签字体大小
        ax.tick_params(which='both', direction='in', top=True, right=True, labelsize=16)
        ax.tick_params(which='minor', length=4)
        ax.tick_params(which='major', length=7)
        # 手动确保轴标签字体大小（如果全局设置未生效）
        ax.xaxis.label.set_size(18)
        ax.yaxis.label.set_size(18)

g.export('joint_new_ds.pdf')

## add fisher

In [11]:
# ['omegabh2','omegach2','theta','tau','logA','ns','omegam','sigma8','S8']

#read
planck_fish = np.array([[ 3.02472812e+10, -2.69892585e+10,  5.67796164e+09,
         5.44639949e+05, -1.40040253e+09, -1.05784155e+09,
         3.59660942e+09,  4.04035934e+09, -5.75015842e+08],
       [-2.69892585e+10,  2.41705642e+10, -5.16707864e+09,
         7.28149628e+02,  1.24243004e+09,  9.38917646e+08,
        -3.20084244e+09, -3.56209167e+09,  4.87584827e+08],
       [ 5.67796164e+09, -5.16707864e+09,  1.41305566e+09,
         1.83769078e+05, -2.23565820e+08, -1.69131096e+08,
         6.14586769e+08,  5.54702893e+08, -3.95875746e+06],
       [ 5.44639949e+05,  7.28149631e+02,  1.83769078e+05,
         1.54601952e+05, -6.85635494e+04, -1.26040295e+04,
        -1.48369814e+05, -1.44648748e+05,  1.38575209e+05],
       [-1.40040253e+09,  1.24243004e+09, -2.23565820e+08,
        -6.85635494e+04,  6.99160695e+07,  5.27844530e+07,
        -1.74466475e+08, -2.12588468e+08,  3.93373407e+07],
       [-1.05784155e+09,  9.38917646e+08, -1.69131096e+08,
        -1.26040295e+04,  5.27844530e+07,  3.99808749e+07,
        -1.31780244e+08, -1.60540099e+08,  2.96860809e+07],
       [ 3.59660942e+09, -3.20084244e+09,  6.14586769e+08,
        -1.48369814e+05, -1.74466475e+08, -1.31780244e+08,
         6.10722977e+08,  6.53352806e+08, -2.17534724e+08],
       [ 4.04035934e+09, -3.56209167e+09,  5.54702893e+08,
        -1.44648748e+05, -2.12588468e+08, -1.60540099e+08,
         6.53352806e+08,  7.73293879e+08, -2.42832089e+08],
       [-5.75015842e+08,  4.87584827e+08, -3.95875746e+06,
         1.38575209e+05,  3.93373407e+07,  2.96860809e+07,
        -2.17534724e+08, -2.42832089e+08,  1.41992902e+08]])

mean = np.array([0.02235975, 0.12020024, 1.04090357, 0.05444509, 3.04473522,
       0.96485744, 0.31655132, 0.8119776 , 0.8340452 ])

In [12]:
def add_matrix_elements(a, b):
    # 创建矩阵a的副本
    result = np.copy(a)
    result[6:8, 6:8] += b
    return result

In [13]:
planck_rsd_fish = add_matrix_elements(planck_fish, fisher_rsd)
planck_rsd_ksz_fish = add_matrix_elements(planck_fish, fisher_rsd_ksz)
planck_rsd_cov = np.linalg.inv(planck_rsd_fish)
planck_rsd_ksz_cov = np.linalg.inv(planck_rsd_ksz_fish)

less1 = planck_rsd_cov[-3:,-3:]
less2 = planck_rsd_ksz_cov[-3:,-3:]


In [14]:
less1, less2

(array([[5.85712869e-05, 1.47395256e-05, 9.23143320e-05],
        [1.47395256e-05, 4.19672324e-05, 6.25004322e-05],
        [9.23143319e-05, 6.25004322e-05, 1.85814624e-04]]),
 array([[ 2.37692309e-05, -1.23550439e-05,  1.86413800e-05],
        [-1.23550439e-05,  1.58539894e-05, -8.33769390e-09],
        [ 1.86413800e-05, -8.33769372e-09,  2.45672351e-05]]))